In [ ]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
from faker import Faker
from passlib.context import CryptContext
from random import randint

load_dotenv()

MSSQL_USER = os.getenv("MSSQL_USER")
MSSQL_PASSWD = os.getenv("MSSQL_PASSWD")
MSSQL_HOST = os.getenv("MSSQL_HOST")
MSSQL_PORT = os.getenv("MSSQL_PORT")

bcrypt = CryptContext(schemes=["bcrypt"], deprecated="auto")

engine = create_engine(f"mssql+pymssql://{MSSQL_USER}:{MSSQL_PASSWD}@{MSSQL_HOST}:{MSSQL_PORT}/todos")
engine

In [ ]:
faker = Faker("pt_BR")

In [ ]:
# setting ansi warnings off to allow truncated strings to be inserted because some faker methods don't have max lenght parameter

In [ ]:
# SEED USERS' TABLE
with engine.begin() as conn:
    query = "SET ANSI_WARNINGS OFF;\nINSERT INTO [todos].[dbo].[users] VALUES\n"
    for i in range(1, 101):  # 100 users
        if i % 10 != 0:
            query += f"({i}, '{faker.email()}', '{faker.user_name()}', '{faker.first_name()}', '{faker.last_name()}', '{bcrypt.hash(faker.password())}', {randint(0, 1)}, '{faker.job()}', '{faker.phone_number()}'),\n"
        else:
            query += f"({i}, '{faker.email()}', '{faker.user_name()}', '{faker.first_name()}', '{faker.last_name()}', '{bcrypt.hash(faker.password())}', {randint(0, 1)}, '{faker.job()}', '{faker.phone_number()}'),\n"
            query = query[:-2] + ";\n" + "SET ANSI_WARNINGS ON;\n"
            conn.execute(text(query))
            query = "SET ANSI_WARNINGS OFF;\nINSERT INTO [todos].[dbo].[users] VALUES\n"
            

In [ ]:
# SEED TODOS' TABLE
with engine.begin() as conn:
    query = "SET ANSI_WARNINGS OFF;\nINSERT INTO [todos].[dbo].[todos] VALUES\n"
    c = 1
    for i in range(1, 101):  # 100 users
        if i % 10 != 0:
            for _ in range(3):  # 3 todos for each user
                query += f"({c}, '{faker.sentence(nb_words=3)}', '{faker.sentence(nb_words=5)}', {randint(1, 5)}, {randint(0, 1)}, {i}),\n"
                c += 1
        else:
            for _ in range(3):
                query += f"({c}, '{faker.sentence(nb_words=3)}', '{faker.sentence(nb_words=5)}', {randint(1, 5)}, {randint(0, 1)}, {i}),\n"
                c += 1
            query = query[:-2] + ";\n" + "SET ANSI_WARNINGS ON;\n"
            conn.execute(text(query))
            query = "SET ANSI_WARNINGS OFF;\nINSERT INTO [todos].[dbo].[todos] VALUES\n"

In [ ]:
# insert test user
with engine.begin() as conn:
    query = f"""
    INSERT INTO [todos].[dbo].[users] VALUES
    (101, 'dev@user.com', 'testuser', 'test', 'user', '{bcrypt.hash("testpass")}', 1, 'tester', '99999999999')
    """
    conn.execute(text(query))